# 1) Imports

In [81]:
import pandas as pd
import numpy as np
import unidecode
import re
import pandas as pd
from geopy.distance import great_circle

# 2) Coleta dos dados

In [82]:
top_muns = pd.read_csv("../Dados/Tabela_final/municipios_potenciais.csv").drop(columns=['Unnamed: 0'])
mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv').drop(columns = 'Unnamed: 0')
agro = pd.read_csv("../Dados/Tabela_final/dados_producao_agricola.csv").drop(columns=['Unnamed: 0'])
solos = pd.read_csv("../Dados/Views/solos_municipios.csv").drop(columns=['Unnamed: 0'])
solos = mun_ser.drop(columns=['LATITUDE', 'LONGITUDE']).merge(solos, on = 'IBGE7').drop(columns=['IBGE7', 'UF'])

# 3) Análise dos dados

### 3.1) Distribuição dos dados

In [85]:
agro

,MUNICIPIO,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD
0,ABAIARA,ALGODAO HERBACEO (EM CAROCO),26.666667,26.666667,2796.333333,214.666667
1,ABAIARA,AMENDOIM (EM CASCA),15.000000,15.000000,978.000000,60.666667
2,ABAIARA,BANANA,26.666667,26.666667,18363.333333,786.333333
3,ABAIARA,CASTANHA DE CAJU,7.000000,7.000000,333.666667,7.333333
4,ABAIARA,FAVA (EM GRAO),37.666667,37.666667,450.000000,86.666667
...,...,...,...,...,...,...
9517,XIQUE-XIQUE,TOMATE,1.000000,1.000000,9333.333333,24.666667
9518,ZABELE,FEIJAO (EM GRAO),193.333333,95.000000,219.000000,87.666667
9519,ZABELE,MANGA,4.000000,4.000000,6666.666667,31.333333
9520,ZABELE,MILHO (EM GRAO),193.333333,95.000000,429.666667,52.333333


### 3.1) Identificando os municípios mais próximos dos municípios potenciais

In [83]:
resultado = {}

for _, row_top in top_muns.iterrows():
    municipio_interesse = row_top['NOME']  
    coordenadas_interesse = (row_top['LATITUDE'], row_top['LONGITUDE'])

    municipios_proximos = {}

    for _, row_ser in mun_ser.iterrows():
        coordenadas_ser = (row_ser['LATITUDE'], row_ser['LONGITUDE'])

        distancia = great_circle(coordenadas_interesse, coordenadas_ser).kilometers

        if distancia <= 50:
            municipios_proximos[row_ser['NOME']] = round(distancia,2)

    resultado[municipio_interesse] = municipios_proximos

resultado

{'ITAPICURU': {'CIPÓ': 39.18,
  'CRISÓPOLIS': 23.38,
  'ITAPICURU': 0.0,
  'NOVA SOURE': 29.68,
  'OLINDINA': 13.0,
  'RIBEIRA DO AMPARO': 36.69,
  'TOBIAS BARRETO': 28.58},
 'MATA GRANDE': {'CANAPI': 14.89,
  'DELMIRO GOUVEIA': 41.68,
  'INHAPI': 12.15,
  'MARAVILHA': 43.53,
  'MATA GRANDE': 0.0,
  "OLHO D'ÁGUA DO CASADO": 44.17,
  'OURO BRANCO': 41.6,
  'PARICONHA': 33.04,
  'SENADOR RUI PALMEIRA': 49.37,
  'INAJÁ': 26.56,
  'ITAÍBA': 39.56,
  'MANARI': 20.34,
  'TACARATU': 45.96},
 'MONTE SANTO': {'CANSANÇÃO': 30.88,
  'EUCLIDES DA CUNHA': 35.51,
  'MONTE SANTO': 0.0,
  'NORDESTINA': 43.77,
  'QUIJINGUE': 37.14},
 'ITAÍBA': {'CANAPI': 27.58,
  'INHAPI': 48.09,
  'MARAVILHA': 32.46,
  'MATA GRANDE': 39.56,
  'OURO BRANCO': 24.65,
  'POÇO DAS TRINCHEIRAS': 42.62,
  'ÁGUAS BELAS': 37.23,
  'BUÍQUE': 45.95,
  'INAJÁ': 46.15,
  'ITAÍBA': 0.0,
  'MANARI': 23.6,
  'TUPANATINGA': 23.39},
 'TUPANATINGA': {'CANAPI': 49.72,
  'OURO BRANCO': 45.7,
  'ÁGUAS BELAS': 47.18,
  'ARCOVERDE': 48.62,
 

### 3.2) Identificando as atividades agrícolas mais exploradas em um raio de 50km do município de interesse

In [84]:
def atividades_agricolas():
    muns_prox = list(resultado['MONTE SANTO'].keys())
    atv_agr_prox = agro[agro['MUNICIPIO'].isin(muns_prox)]
    atv_agr_prox = atv_agr_prox[['PRODUTO', 'AREA_PLANTADA', 'VALOR_PROD']].groupby('PRODUTO').sum().reset_index().sort_values(by='VALOR_PROD', ascending = False)
    atv_agr_prox['REND_AREA'] = atv_agr_prox['VALOR_PROD']/atv_agr_prox['AREA_PLANTADA']
    solos_prox = solos[solos['NOME'].isin(muns_prox)].drop(columns=['NOME'])
    solos_prox = solos_prox.groupby('SOLO').sum().reset_index().sort_values(by='AREA_TOTAL', ascending = False)
    print('Total')
    display(atv_agr_prox)
    display(solos_prox)
    
    for mun in resultado['MONTE SANTO']:
        atv_agr_prox = agro[agro['MUNICIPIO']==mun]
        atv_agr_prox = atv_agr_prox[['PRODUTO', 'AREA_PLANTADA', 'VALOR_PROD']].groupby('PRODUTO').sum().reset_index().sort_values(by='VALOR_PROD', ascending = False)
        atv_agr_prox['REND_AREA'] = atv_agr_prox['VALOR_PROD']/atv_agr_prox['AREA_PLANTADA']
        
        solos_prox = solos[solos['NOME']==mun].drop(columns=['NOME'])
        solos_prox = solos_prox.groupby('SOLO').sum().reset_index().sort_values(by='AREA_TOTAL', ascending = False)
        
        if atv_agr_prox.shape[0] == 0:
            pass
        else:
            print(mun)
            display(atv_agr_prox)
            display(solos_prox)
        
    
atividades_agricolas()

Total


,PRODUTO,AREA_PLANTADA,VALOR_PROD,REND_AREA
4,MILHO (EM GRAO),42333.333333,96231.000000,2.273173
2,FEIJAO (EM GRAO),40588.333333,70331.666667,1.732805
3,MANDIOCA,3591.666667,20512.666667,5.711183
1,CASTANHA DE CAJU,1054.333333,756.000000,0.717041
0,BATATA-DOCE,3.333333,41.666667,12.500000


,SOLO,AREA_TOTAL
6,SXe - Planossolos Haplicos Eutroficos,546281.678800
5,RRe - Neossolos Regoliticos Eutroficos,138945.991192
0,CXve - Cambissolos Haplicos Ta Eutroficos,63653.886201
2,PVAd - Argissolos Vermelho-Amarelos Distroficos,53856.149036
4,RQo - Neossolos Quartzarenicos Orticos,41773.600707
3,RLd - Neossolos Litolicos Distroficos,25751.551286
1,LAd - Latossolos Amarelos Distroficos,1034.400607


EUCLIDES DA CUNHA


,PRODUTO,AREA_PLANTADA,VALOR_PROD,REND_AREA
2,FEIJAO (EM GRAO),20066.666667,56344.000000,2.807841
4,MILHO (EM GRAO),14000.000000,52795.666667,3.771119
3,MANDIOCA,3000.000000,15999.333333,5.333111
1,CASTANHA DE CAJU,1000.000000,710.666667,0.710667
0,BATATA-DOCE,3.333333,41.666667,12.500000


...

,PRODUTO,AREA_PLANTADA,VALOR_PROD,REND_AREA
2,MILHO (EM GRAO),2166.666667,1571.333333,0.725231
1,MANDIOCA,96.666667,130.666667,1.351724
0,FEIJAO (EM GRAO),1833.333333,68.000000,0.037091


,SOLO,AREA_TOTAL
1,SXe - Planossolos Haplicos Eutroficos,40353.565744
0,RRe - Neossolos Regoliticos Eutroficos,6038.228454


QUIJINGUE


,PRODUTO,AREA_PLANTADA,VALOR_PROD,REND_AREA
3,MILHO (EM GRAO),17166.666667,7692.333333,0.448097
1,FEIJAO (EM GRAO),9671.666667,5694.333333,0.588764
2,MANDIOCA,290.000000,2805.000000,9.672414
0,CASTANHA DE CAJU,49.333333,41.333333,0.837838


,SOLO,AREA_TOTAL
3,SXe - Planossolos Haplicos Eutroficos,103958.956936
2,RQo - Neossolos Quartzarenicos Orticos,28659.404691
0,CXve - Cambissolos Haplicos Ta Eutroficos,7364.840152
1,LAd - Latossolos Amarelos Distroficos,1034.400607


In [80]:
top_muns[['NOME', '']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 60 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IBGE7                15 non-null     int64  
 1   UF                   15 non-null     object 
 2   NOME                 15 non-null     object 
 3   LATITUDE             15 non-null     float64
 4   LONGITUDE            15 non-null     float64
 5   DIST_EF_PUB          15 non-null     float64
 6   DIST_EM_PUB          15 non-null     float64
 7   IDEB_AI              15 non-null     float64
 8   IDEB_AF              15 non-null     float64
 9   DOCSUP_EF_PUB        15 non-null     float64
 10  DOCSUP_EM_PUB        15 non-null     float64
 11  TXNASC7C             15 non-null     float64
 12  TXNBAIXOP            15 non-null     float64
 13  PINTERSAP            15 non-null     float64
 14  PINTERDRSAI          15 non-null     float64
 15  REN_PIBPC_D          15 non-null     float